In [ ]:
!pip install "dlt[bigquery]"

In [ ]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient

In [ ]:
client = RESTClient(base_url="https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/")

In [ ]:
response = client.get("green_tripdata_2019-01.csv.gz")

In [ ]:
!gunzip green_tripdata_2019-01.csv.gz

In [ ]:
from dlt.sources.filesystem import filesystem, read_csv

In [ ]:
!pip3 install pandas

In [ ]:
import dlt
import requests
import gzip
import io

# Define a function to extract data from a remote .csv.gz file
def fetch_csv_gz_from_remote(url):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with gzip.GzipFile(fileobj=io.BytesIO(response.content)) as gz_file:
        return gz_file.read().decode('utf-8')

# Define a dlt pipeline
pipeline = dlt.pipeline(
    pipeline_name="csv_gz_to_bigquery",
    destination="bigquery",  # Use BigQuery as the destination
    dataset_name="your_dataset",  # BigQuery dataset name
)

# URL to the remote .csv.gz file
csv_gz_url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-01.csv.gz"

# Fetch and load the data
data = fetch_csv_gz_from_remote(csv_gz_url)
# Parse the CSV data (assuming it's comma-separated)
rows = [line.split(",") for line in data.splitlines()]

# Load the data into BigQuery
load_info = pipeline.run(rows, table_name="your_table")
print(load_info)

In [ ]:
from dlt.common.storages.fsspec_filesystem import FileItemDict
from dlt.common.typing import TDataItems

@dlt.transformer(standalone=True)
def read_gz(items):
    import pandas as pd
    # Iterate through each file item.
    for file_obj in items:
        # Open the file object.
        with file_obj.open() as file:
            # Parse the file to dict records.
            yield pd.read_csv(file)

In [ ]:
!pip3 install "dlt[bigquery]"
import dlt
from dlt.sources.filesystem import filesystem, read_csv

files = filesystem(extract_content=True, bucket_url="https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green", file_glob="green_tripdata_2019-01.csv.gz")
filesystem_pipe = files | read_gz()


pipeline = dlt.pipeline(pipeline_name="testing_taxi1", dataset_name="testing_taxi3", destination="bigquery")

load_info = pipeline.run(filesystem_pipe.with_name("testing_taxi2"))
load_info = pipeline.run(filesystem_pipe.with_name("testing_taxi2"))

print(load_info)
print(pipeline.last_trace.last_normalize_info)


In [ ]:
"https://www.stats.govt.nz/assets/Uploads/Annual-enterprise-survey/Annual-enterprise-survey-2023-financial-year-provisional/Download-data/annual-enterprise-survey-2023-financial-year-provisional-size-bands.csv"

In [ ]:
import dlt
from dlt.sources.filesystem import filesystem, read_csv

files = filesystem(extract_content=True, bucket_url="https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green",
                    file_glob="green_tripdata_2019-01.csv.gz")
filesystem_pipe = files | read_gz()


pipeline = dlt.pipeline(pipeline_name="testing_taxi1", dataset_name="testing_taxi3", destination="bigquery")

load_info = pipeline.run(filesystem_pipe.with_name("testing_taxi2"))
load_info = pipeline.run(filesystem_pipe.with_name("testing_taxi2"))

print(load_info)
print(pipeline.last_trace.last_normalize_info)


In [ ]:
!pip3 install google-cloud-bigquery-storage

In [4]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
import requests
import pandas as pd

github_client = RESTClient(base_url="http://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/")


!pip3 install certifi
'''
@dlt.resource
def hubspot():
    for row in pd.read_csv(github_client.base_url + "green_tripdata_2019-01.csv.gz", 
                           compression='gzip', header=0, sep=' '):
        yield row

'''
'''
@dlt.resource
def hubspot():
    for row in pd.read_table(r'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-01.csv.gz'):
        yield row
'''

!pip3 install pip-system-certs==4.0.0

!source '/Applications/Python 3.12/Install Certificates.command'

base_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/'
type_of_taxi_= ['green', 'yellow', 'fhv']
years_for_query = ['2019', '2020']
months_for_query = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
file_name = f'green_tripdata_2019-01.csv.gz'

@dlt.resource
def hubspot(taxi, year, month):
    type_of_taxi = taxi
    url = base_url + type_of_taxi + '/' + type_of_taxi + '_tripdata_' + year + '-' + month + '.csv.gz'
    print('downloading ' + url)
    s = requests.get(url, verify=True)
    if taxi=='green' or taxi=='yellow':
        c = pd.read_csv(url, compression='gzip',
                    dtype={
                'VendorID': 'Int64',
                'lpep_pickup_datetime': 'str',
                'lpep_dropoff_datetime': 'str',
                'store_and_fwd_flag': 'str',
                'RatecodeID': 'Int64',
                'PULocationID': 'str',
                'DOLocationID': 'str',
                'passenger_count': 'Int64',
                'trip_distance': 'float64',
                'fare_amount': 'float64',
                'extra': 'float64',
                'mta_tax': 'float64',
                'tip_amount': 'float64',
                'tolls_amount': 'float64',
                'ehail_fee': 'float64',
                'improvement_surcharge': 'float64',
                'total_amount': 'float64',
                'payment_type': 'Int64',
                'trip_type': 'Int64',
                'congestion_surcharge': 'float64'}
                        )
    if taxi=='fhv':
        c = pd.read_csv(url, compression='gzip',
                    dtype={
                'dispatching_base_num': 'str',
                'pickup_datetime': 'str',
                'dropOff_datetime': 'str',
                'PUlocationID': 'str',
                'DOlocationID': 'str',
                'SR_Flag': 'str',
                'Affiliated_base_number': 'str'})

        c.pickup_datetime =  pd.to_datetime(c.pickup_datetime)
        c.dropOff_datetime = pd.to_datetime(c.dropOff_datetime) 
        c.PUlocationID = c.PUlocationID.astype('str')
        c.DOlocationID = c.DOlocationID.astype('str')  

    if taxi=='green':
        c.lpep_pickup_datetime =  pd.to_datetime(c.lpep_pickup_datetime)
        c.lpep_dropoff_datetime = pd.to_datetime(c.lpep_dropoff_datetime)  

    elif taxi=='yellow':
        c.tpep_pickup_datetime =  pd.to_datetime(c.tpep_pickup_datetime)
        c.tpep_dropoff_datetime = pd.to_datetime(c.tpep_dropoff_datetime) 
    yield c   

@dlt.resource
def hubspot_yellow():
    type_of_taxi = type_of_taxi_[1]
    for year in years_for_query:
        for month in months_for_query:
            url = base_url + type_of_taxi + '/' + type_of_taxi + '_tripdata_' + year + '-' + month + '.csv.gz'
            print('downloading ' + url)
            s = requests.get(url, verify=True)
            c = pd.read_csv(url, compression='gzip')
            c.lpep_pickup_datetime =  pd.to_datetime(c.lpep_pickup_datetime)
            c.lpep_dropoff_datetime = pd.to_datetime(c.lpep_dropoff_datetime)   
            yield c  

@dlt.resource
def hubspot_fhv():
    type_of_taxi = type_of_taxi_[2]
    year = years_for_query[0]
    for month in months_for_query:
        url = base_url + type_of_taxi + '/' + type_of_taxi + '_tripdata_' + year + '-' + month + '.csv.gz'
        print('downloading ' + url)
        s = requests.get(url, verify=True)
        c = pd.read_csv(url, compression='gzip')              
        c.pickup_datetime = pd.to_datetime(c.pickup_datetime)
        c.dropoff_datetime = pd.to_datetime(c.dropoff_datetime)
        yield c
'''
    url = r"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-01.csv.gz"
    s = requests.get(url, verify=True)
    c = pd.read_csv(url, compression='gzip')
    c.lpep_pickup_datetime =  pd.to_datetime(c.lpep_pickup_datetime)
    c.lpep_dropoff_datetime = pd.to_datetime(c.lpep_dropoff_datetime)
    yield c
'''  

pipeline = dlt.pipeline(
    pipeline_name="github_issues",
    destination="bigquery",
    dataset_name="github_data"
)

#for year in years_for_query:
#    for month in months_for_query:
#        load_info = pipeline.run(hubspot('green', year, month), table_name="green_taxi")

#for year in years_for_query:
#    for month in months_for_query:
#        load_info = pipeline.run(hubspot('yellow', year, month), table_name="yellow_taxi")

year = '2019'
for month in months_for_query:
    load_info = pipeline.run(hubspot('fhv', year, month), table_name="fhv")


print(load_info)


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
 -- pip install --upgrade certifi

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
 -- removing any existing file or link
 -- creating symlink to certifi certificate bundle
 -- setting permissions
 -- update complete
downloading https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-01.csv.gz


2025-02-23 23:38:07,028|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'tpep_pickup_datetime'. in table 'yellow_taxi'.
2025-02-23 23:38:07,029|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'tpep_dropoff_datetime'. in table 'yellow_taxi'.
2025-02-23 23:38:07,029|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'pickup_datetime'. in table 'fhv'.
2025-02-23 23:38:07,029|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these fl

downloading https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-02.csv.gz


2025-02-23 23:49:51,095|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'tpep_pickup_datetime'. in table 'yellow_taxi'.
2025-02-23 23:49:51,095|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'tpep_dropoff_datetime'. in table 'yellow_taxi'.
2025-02-23 23:49:51,095|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'pickup_datetime'. in table 'fhv'.
2025-02-23 23:49:51,096|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these fl

downloading https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-03.csv.gz


...

downloading https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-11.csv.gz


2025-02-24 00:07:06,907|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'tpep_pickup_datetime'. in table 'yellow_taxi'.
2025-02-24 00:07:06,907|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'tpep_dropoff_datetime'. in table 'yellow_taxi'.
2025-02-24 00:07:06,907|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'pickup_datetime'. in table 'fhv'.
2025-02-24 00:07:06,907|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these fl

downloading https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-12.csv.gz


2025-02-24 00:08:57,858|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'tpep_pickup_datetime'. in table 'yellow_taxi'.
2025-02-24 00:08:57,859|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'tpep_dropoff_datetime'. in table 'yellow_taxi'.
2025-02-24 00:08:57,859|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'pickup_datetime'. in table 'fhv'.
2025-02-24 00:08:57,859|[WARNING]|49993|8438220864|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these fl

Pipeline github_issues load step completed in 1 minute and 34.48 seconds
1 load package(s) were loaded to destination bigquery and into dataset github_data
The bigquery destination used de-zoomcamp-module-1@lofty-hybrid-448518-c8.iam.gserviceaccount.com@lofty-hybrid-448518-c8 location to store data
Load package 1740348517.5967438 is LOADED and contains no failed jobs


In [ ]:
!pip3 pandas --version

In [ ]:
import pandas as pd
pd.__version__